# Is park weather associated with visitor counts for 2022?

Hypothesis

Null:  Weather and visitation are not correlated.

Alternate:  Parks with average summer temperatures between 65-85 degrees have more visitors.

In [36]:
#Dependencies
import pandas as pd
import json
import requests
import csv
import matplotlib
import scipy.stats as st
from configure import VC_API_key

In [38]:
# Pull visitor and park info from resources, especially looking for lat and long and at the weather column.

# Dan's code
from configure import NPS_API_key
from pathlib import Path
import re
from pandas import json_normalize 

park_visitors_csv_file = Path("../Resources/national_park_visitors.csv")

#Read the csv and create variable called park_visitors_data
park_visitors_data= pd.read_csv(park_visitors_csv_file)

#Make it into a dataframe
park_visitors_df = pd.DataFrame(park_visitors_data)

#Establish nps endpoint - with api key
endpoint = f"https://developer.nps.gov/api/v1/parks?limit=600&api_key={NPS_API_key}"

#turn the result into a json file
parks_data = requests.get(endpoint).json()

#Normalize data so it all fits into a dataframe
normalized_parks_data = json_normalize(parks_data, 'data')

#Drop the columns we dont need
all_parks_df = pd.DataFrame(normalized_parks_data).drop(columns=[
'id', 
    'url', 
    'latLong', 
    'directionsInfo', 
    'directionsUrl', 
    'addresses', 
    'images', 
    'contacts.phoneNumbers',
    'contacts.emailAddresses',
    
])

#sort by only those named national park or national park and preserve
national_parks_df = all_parks_df.loc[(all_parks_df['designation'] == "National Park") | (all_parks_df['designation'] == "National Park & Preserve")]

#Clean up the " NP" Part of the Park Name
park_visitors_df_clean = park_visitors_df.replace(' NP','', regex=True)

#rename park_visitors_df columns
park_visitors_df_clean.columns = ["Park Name", "Rank", "Visitors", "Percent Of Total"]

#create a copy to start renaming columns
national_parks_df_clean = national_parks_df.copy()

#rename columns
national_parks_df_clean.rename(columns={
    'fullName':'Full Name',
    'parkCode':'Park Code',
    'description':'Description',
    'latitude':'Latitude',
    'longitude':'Longitude',
    'activities':'Activities',
    'topics':'Topics',
    'states':'States',
    'entranceFees':'Entrance Fees',
    'entrancePasses':'Entrance Passes',
    'fees':'Fees',
    'operatingHours':'Operating Hours',
    'weatherInfo':'Weather Info',
    'designation':'Designation',
    'name':'Park Name'
}, inplace = True)

#merge visitor df and park df
combined_park_data_df = park_visitors_df_clean.merge(national_parks_df_clean, on="Park Name", how='left')

print(len(combined_park_data_df))
combined_park_data_df

63


Park Name  Rank    Visitors Percent Of Total  \
0             Great Smoky Mountains     1  12,937,633           14.59%   
1                      Grand Canyon     2   4,732,101            5.34%   
2                              Zion     3   4,692,417            5.29%   
3                    Rocky Mountain     4   4,300,424            4.85%   
4                            Acadia     5   3,970,260            4.48%   
..                              ...   ...         ...              ...   
58                      Isle Royale    59      25,454            0.03%   
59                Lake Clark & PRES    60      18,187            0.02%   
60                     Kobuk Valley    61      16,925            0.02%   
61       Gates of the Arctic & PRES    62       9,457            0.01%   
62  National Park of American Samoa    63       1,887            0.00%   

                              Full Name Park Code  \
0   Great Smoky Mountains National Park      grsm   
1            Grand Canyon National Park      grca   
2                    Zion National Park      zion   
3          Rocky Mountain National Park      romo   
4                  Acadia National Park      acad   
..                                  ...       ...   
58            Isle Royale National Park      isro   
59                                  NaN       NaN   
60           Kobuk Valley National Park      kova   
61                                  NaN       NaN   
62                                  NaN       NaN   

                                          Description       Latitude  \
0   Ridge upon ridge of forest straddles the borde...    35.60116374   
1   Grand Canyon National Park, in Northern Arizon...  36.0001165336   
2   Follow the paths where native people and pione...    37.29839254   
3   Rocky Mountain National Park's 415 square mile...     40.3556924   
4   Acadia National Park protects the natural beau...      44.409286   
..                                                ...            ...   
58  Explore a rugged, isolated island far from our...    48.01145819   
59                                                NaN            NaN   
60  Caribou, sand dunes, the Kobuk River, Onion Po...    67.35631336   
61                                                NaN            NaN   
62                                                NaN            NaN   

         Longitude                                         Activities  \
0     -83.50818326  [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...   
1   -112.121516363  [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...   
2     -113.0265138  [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...   
3     -105.6972879  [{'id': '5F723BAD-7359-48FC-98FA-631592256E35'...   
4       -68.247501  [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...   
..             ...                                                ...   
58    -88.82780657  [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...   
59             NaN                                                NaN   
60    -159.2002293  [{'id': '071BA73C-1D3C-46D4-A53C-00D5602F7F0E'...   
61             NaN                                                NaN   
62             NaN                                                NaN   

                                               Topics States  \
0                                                  []  NC,TN   
1   [{'id': '69693007-2DF2-4EDE-BB3B-A25EBA72BDF5'...     AZ   
2   [{'id': '69693007-2DF2-4EDE-BB3B-A25EBA72BDF5'...     UT   
3   [{'id': '69693007-2DF2-4EDE-BB3B-A25EBA72BDF5'...     CO   
4   [{'id': '00F3C3F9-2D67-4802-81AE-CCEA5D3BA370'...     ME   
..                                                ...    ...   
58  [{'id': '00F3C3F9-2D67-4802-81AE-CCEA5D3BA370'...     MI   
59                                                NaN    NaN   
60  [{'id': '7F81A0CB-B91F-4896-B9A5-41BE9A54A27B'...     AK   
61                                                NaN    NaN   
62                                                NaN 

In [26]:
#test weather data
#Tennessee Entrance (Sugarlands Visitor Center):
#Latitude: 35.6754° N
#Longitude: -83.4341° W

#North Carolina Entrance (Oconaluftee Visitor Center):
#Latitude: 35.5204° N
#Longitude: -83.0466° W

lat = 35.6
lon = -83.2

In [27]:
# #Set up url for open weather api calls
# #Create lists of unix dates for weather url from https://www.unixtimestamp.com/index.php
# may_start = 1651388400  #May 1
# may_end = 1651993200  #May 8
# jun_start = 1653807600  #May 29
# jun_end = 1654412400  #June 5
# jul_start = 1656226800  #June 26
# jul_end = 1656745200  #July 2
# aug_start = 1659250800  #July 31
# aug_end = 1659855600 #August 7
# sep_start = 1661670000  #Augst 28
# sep_end = 1662274800  #September 4


# # Set Params:
# lat = 35.6  #GSMNP
# lon = -83.2
# # start = [may_start, jun_start, jul_start, aug_start, sep_start]
# # end = [may_end, jun_end, jul_end, aug_end, sep_end]
# #test may
# start = may_start
# end = may_end

# # set up a parameters dictionary
# params = {
#     "lat":lat,
#     "lon":lon,
#     "start":start,
#     "end":end,
#     "units":"imperial",
#     "appid":weather_api_key
# }

# # Set base URL
# # url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&units=imperial&appid={weather_api_key}"

# base_url = "https://history.openweathermap.org/data/2.5/history/city?"

# weather_data = requests.get(base_url, params).json()

# print(json.dumps(weather_data,indent = 4, sort_keys = True))


{
    "cod": 401,
    "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."
}


In [ ]:
#Visual Crossing

url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C%20{lon}/2022-05-01/2022-09-30?unitGroup=us&elements=datetime%2Cname%2Ctempmax%2Ctempmin%2Cprecip%2Ccloudcover&include=days%2Cobs&key={VC_API_key}&contentType=json&forecastBasisDate=2022-05-01"